In [4]:
# getting all libraires


import os
import tarfile
import urllib

In [5]:
download_root = "https://raw.githubusercontent.com/ageron/handson-ml2/master"
housing_path = os.path.join("datasets","housing")
housing_url = download_root + "datasets/housing/housing.tgz"